## 七月在线机器学习实训营三期第七周(机器学习-模型调优与融合)考试
#### 考试说明:
- 起止时间：请同学在2018年3月23日至3月25日期间完成，最晚提交时间本周日（3月25日24时之前）结束，<b>逾期不接受补考,该考试分数计入平时成绩</b>
- 考试方式：请同学<font color=red><b>拷贝</b></font>该试卷至自己姓名的目录后，将文件更名为同学姓名拼音-exam7后，进行作答。例如wangwei-exam7
- 提交格式：请同学新建自己姓名全拼的文件夹，将该试卷，数据文件，zip文件等相关考试文件，放置此目录下。将该目录<b>移动</b>至/0.Teacher/Exam/7/目录下
- 注意事项：为确保同学们真正了解自身对本周课程的掌握程度，<font color=red><b>请勿翻阅，移动，更改</b></font>其它同学试卷。如发现按0分处理
- 请同学在下方同学姓名处填写自己的姓名，批改人和最终得分不用填写

- 同学姓名:<u>_思浩_</u>  
- 批改人： David
- 最终得分:97

<center><h1>####答卷开始####</h1></center>
<hr>

## 简答题(共6题，1，2题每题5分，后4题每题10分，共计50分)

- note:47分

In [ ]:
训练集：有标签，训练集是用来训练模型参数的数据。
验证集：有标签，用来评估模型训练的好坏。
测试集：无标签，模型训练好以后，用来预测最后的结果。

In [ ]:
利用d_train = xgb.DMatrix(data, label)将数据转换为libsvm格式。
101:1.2  每行代表一个样本 ‘:’前面的是索引，后面的是值。

#### 3. 对于自动特征选择，通常有哪三类方法？试写出每类方式的思想，以及在sklearn中的代码实现（面试题）

In [56]:
1.单变量分析
    是分别单独的计算每个变量的某个统计指标，根据该指标来判断哪些指标重要，剔除那些不重要的指标。
2.基于模型的特征选择
    模型内部自动选择，训练基模型，选择权值系数较高的特征。
3.逐步特征删除Recursive feature elimination (RFE)
    递归特征消除的主要思想是反复的构建模型（如SVM或者回归模型）然后选出最好的（或者最差的）的特征（可以根据系数来选），把选出来的特征放到
一边，然后在剩余的特征上重复这个过程，直到所有特征都遍历了。这个过程中特征被消除的次序就是特征的排序。

代码实现：
1.单变量
from sklearn.feature_selection import SelectPercentile
#该接口函数默认使用f_classif评估；percentile=50，保留前50%的特征
select = SelectPercentile(percentile=50)
select.fit(X_train, y_train)
# transform training set:
X_train_selected = select.transform(X_train)
2.模型内部选择：
from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import RandomForestClassifier
select = SelectFromModel(RandomForestClassifier(n_estimators=100, random_state=42), threshold="median")
select.fit(X_train, y_train)
X_train_l1 = select.transform(X_train)
3.递归特征消除：
from sklearn.feature_selection import RFE
select = RFE(RandomForestClassifier(n_estimators=100, random_state=42), n_features_to_select=40)
select.fit(X_train, y_train)
X_train_rfe = select.transform(X_train)
X_test_rfe = select.transform(X_test)

SyntaxError: invalid syntax (<ipython-input-56-dc0c2ac507d4>, line 1)

#### 4.网格搜索交叉验证的作用是什么，并简述网格搜索交叉验证是如何运行的？ 

In [ ]:
用来选取模型的表现最优秀的超参数。
网格搜先对输入不同的超参数待选值，遍历各种组合；然后在每组参数下交叉验证将数据按一定比例分成训练集和验证集，循环分组训练然后验证，综合评分然后评价每组超参数的效果，
选取最优超参数组。。
grid_search = GridSearchCV(SVC(), param_grid, cv=5)

#### 5.工业界上模型整合有三大类方式？试简述每类方式其思想？（面试题）

In [ ]:
1.多数表决融合   voting,Bagging,随机森林/Random forest；思想;结和多数人的想法作决定。
2.staking/blending 思想：将多个模型训练结果当做输入特征，然后再次进行训练。
3. Boosting  思想：一万小时定律，对一个模型强化学习，深入学习，效果会有所提高。

#### 6.  我们可以将xgboost的中众多参数分类为哪三类？请写出哪些参数可以用什么方式用来控制过拟合？

In [ ]:
1.分为通用参数、集成参数、任务参数
2.集成参数：eta：学习步长，设置小一些，可以防止过拟合。
            gamma:调大一点防止过拟合，防止树生长过深，样本切分过细过拟合。
            max_depth：树深，设小一些，防止过拟合，在一定意义效果上和gamme类似。
            min_child_weight：设大一些，防止过拟合，叶子结点最小样本数。
            subsample：表示观测的子样本的比率，将其设置为0.5意味着xgboost将随机抽取一半观测用于数的生长，这将有助于防止过拟合现象
            colsample_bytree ：表示用于构造每棵树时变量的子样本比率，将其设置为0.5用于随机选取样本，防止过拟合，colsample_bylevel类似。
            
            

## 实验题(共2题，每题25分，共计50分)

- note: 50

### 1. 使用XGBoost内置方式，导入iris数据完成分类问题（要求以不同参数设置xgboost运行并比对），最后给出实验总结

In [1]:
import xgboost as xgb
from sklearn.datasets import load_iris

In [3]:
iris = load_iris()
X = iris.data
Y = iris.target
d_train = xgb.DMatrix(data=X[:120,:], label=Y[:120])
d_test = xgb.DMatrix(data=X[120:,:], label=Y[120:])

In [4]:
watch_list = [(d_test,'eval'), (d_train,'train')]

In [5]:
#参数设定
param = {'max_depth':3, 'eta':0.01, 'objective':'multi:softmax', 'num_class':3}

num_round = 10
bst = xgb.train(param, d_train, num_round, watch_list)

[0]	eval-merror:0.266667	train-merror:0.016667
[1]	eval-merror:0.266667	train-merror:0.016667
[2]	eval-merror:0.266667	train-merror:0.016667
[3]	eval-merror:0.266667	train-merror:0.016667
[4]	eval-merror:0.266667	train-merror:0.016667
[5]	eval-merror:0.266667	train-merror:0.016667
[6]	eval-merror:0.266667	train-merror:0.016667
[7]	eval-merror:0.266667	train-merror:0.016667
[8]	eval-merror:0.266667	train-merror:0.016667
[9]	eval-merror:0.266667	train-merror:0.016667


In [8]:
#参数设定2
param = {'max_depth':2, 'eta':0.1, 'objective':'multi:softmax', 'num_class':3}
num_round = 10
bst = xgb.train(param, d_train, num_round, watch_list)
#eta从0.01改为0.1，输出没有变化，eta表示第n颗树累加时的和前面n-1颗树组成模型对于训练y差异的学习快慢，从上面数据看出，第1颗后已经不再
#继续学习，模型已成形，改eta没什么用。
#将max_depth从3改到4，没什么变化，说明3已经是比较好的树深度。改为2时，输出如下，说明树深为2时，前几棵树学习的不够。
#总的来说数据较简单，模型学习较快

[0]	eval-merror:0.166667	train-merror:0.025
[1]	eval-merror:0.266667	train-merror:0.025
[2]	eval-merror:0.266667	train-merror:0.025
[3]	eval-merror:0.266667	train-merror:0.025
[4]	eval-merror:0.266667	train-merror:0.016667
[5]	eval-merror:0.266667	train-merror:0.025
[6]	eval-merror:0.266667	train-merror:0.025
[7]	eval-merror:0.266667	train-merror:0.025
[8]	eval-merror:0.266667	train-merror:0.025
[9]	eval-merror:0.266667	train-merror:0.025


### 2. 使用XGBoost的sklearn接口，对KaggleCredit2数据完成信用卡欺诈项目的建模及分析（要求以不同参数设置xgboost运行并比对），最后给出实验总结

- KaggleCredit2数据文件 位于/home/sxy-s3/0.Teacher/Exam/KaggleCredit2.csv.zip，请勿复制或移动该文件

In [51]:
#数据导入
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', 500)
import zipfile
with zipfile.ZipFile('/home/sxy-s3/0.Teacher/Exam/KaggleCredit2.csv.zip', 'r') as z:
    f = z.open('KaggleCredit2.csv')
    data = pd.read_csv(f, index_col=0)
data.head()

,SeriousDlqin2yrs,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents
0,1,0.766127,45.0,2.0,0.802982,9120.0,13.0,0.0,6.0,0.0,2.0
1,0,0.957151,40.0,0.0,0.121876,2600.0,4.0,0.0,0.0,0.0,1.0
2,0,0.658180,38.0,1.0,0.085113,3042.0,2.0,1.0,0.0,0.0,0.0
3,0,0.233810,30.0,0.0,0.036050,3300.0,5.0,0.0,0.0,0.0,0.0
4,0,0.907239,49.0,1.0,0.024926,63588.0,7.0,0.0,1.0,0.0,0.0


In [52]:
#数据切割
data.dropna(inplace=True)
from sklearn.model_selection import train_test_split
y = data['SeriousDlqin2yrs']
X = data.drop('SeriousDlqin2yrs', axis=1)
#X, y = data.ix[:, 1:].values, data.ix[:, 0].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=0)#随机选择25%作为测试集，剩余作为训练集
X_train.shape

(81486, 10)

In [53]:
#模型参数设置
model = xgb.XGBClassifier(max_depth=3, learning_rate=0.01, n_estimators=10, objective='binary:logistic')
#训练
model.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.01, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=10,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [54]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score
import sklearn
ypred =model.predict(X_test)
auc=model.score(X_test, y_test)
pre=sklearn.metrics.precision_score( y_test, ypred)#	Compute the precision
recall=sklearn.metrics.recall_score( y_test, ypred)#	Compute the recall
print('Accuracy: %.6f percision: %.6f recall: %.6f' % (auc,pre,recall))
cnf_matrix_lr=confusion_matrix(y_test, ypred)
cnf_matrix_lr

Accuracy: 0.935498 percision: 0.527586 recall: 0.171525


/usr/local/lib/python3.5/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/usr/local/lib/python3.5/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


array([[25104,   274],
       [ 1478,   306]])

In [45]:
#更改参数1 
#-改树深4 1的准确率上升，召回率下降；
#Accuracy: 0.937228 percision: 0.583864 recall: 0.154148
#-改树深6 1的准确率上升，召回率下降；
#Accuracy: 0.935645 percision: 0.538136 recall: 0.142377
#-改树深8 1的准确率下降，召回率相比6时提高一点；
#Accuracy: 0.935498 percision: 0.531250 recall: 0.152466
#模型参数设置
model = xgb.XGBClassifier(max_depth=5, learning_rate=0.01, n_estimators=10, objective='binary:logistic')
#训练
model.fit(X_train, y_train)
ypred =model.predict(X_test)
auc=model.score(X_test, y_test)
pre=sklearn.metrics.precision_score( y_test, ypred)#	Compute the precision
recall=sklearn.metrics.recall_score( y_test, ypred)#	Compute the recall
print('Accuracy: %.6f percision: %.6f recall: %.6f' % (auc,pre,recall))
cnf_matrix_lr=confusion_matrix(y_test, ypred)
cnf_matrix_lr

Accuracy: 0.937228 percision: 0.583864 recall: 0.154148


/usr/local/lib/python3.5/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/usr/local/lib/python3.5/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


array([[25182,   196],
       [ 1509,   275]])

In [62]:
#更改参数2（n_estimators）
#-改n_estimators5 1的准确率上升，召回率下降；
#Accuracy: 0.937118 percision: 0.603261 recall: 0.124439
#-改n_estimators20 1的准确率上升，召回率上升；
#Accuracy: 0.937265 percision: 0.583682 recall: 0.156390
#-改n_estimators20 1的准确率上升，召回率上升；
#Accuracy: 0.937302 percision: 0.584200 recall: 0.157511
#-改n_estimators30 1的准确率下降，召回率上升；
#Accuracy: 0.937008 percision: 0.570600 recall: 0.165359
#模型参数设置
model = xgb.XGBClassifier(max_depth=5, learning_rate=0.01, n_estimators=25, objective='binary:logistic')
#训练
model.fit(X_train, y_train)
ypred =model.predict(X_test)
auc=model.score(X_test, y_test)
pre=sklearn.metrics.precision_score( y_test, ypred)#	Compute the precision
recall=sklearn.metrics.recall_score( y_test, ypred)#	Compute the recall
print('Accuracy: %.6f percision: %.6f recall: %.6f' % (auc,pre,recall))
cnf_matrix_lr=confusion_matrix(y_test, ypred)
cnf_matrix_lr

Accuracy: 0.937339 percision: 0.584016 recall: 0.159753


/usr/local/lib/python3.5/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/usr/local/lib/python3.5/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


array([[25175,   203],
       [ 1499,   285]])

In [65]:
#更改参数 -网格搜索
from sklearn.model_selection import KFold, train_test_split, GridSearchCV
print("Parameter optimization")
clf = GridSearchCV(model,
                   {'max_depth': [2,3,5],
                    'learning_rate':[0.01,0.1],
                    'n_estimators': [10,20]},
                   scoring='recall',
                   verbose=1)
clf.fit(X_train, y_train)
print(clf.best_score_)
print(clf.best_params_)

Parameter optimization
Fitting 3 folds for each of 12 candidates, totalling 36 fits


/usr/local/lib/python3.5/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/usr/local/lib/python3.5/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/usr/local/lib/python3.5/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/usr/local/lib/python3.5/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this wi

/usr/local/lib/python3.5/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/usr/local/lib/python3.5/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/usr/local/lib/python3.5/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/usr/local/lib/python3.5/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this wi

/usr/local/lib/python3.5/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/usr/local/lib/python3.5/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/usr/local/lib/python3.5/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/usr/local/lib/python3.5/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this wi

0.15859746951334647
{'n_estimators': 20, 'learning_rate': 0.01, 'max_depth': 3}


In [66]:
model = xgb.XGBClassifier(max_depth=3, learning_rate=0.01, n_estimators=20, objective='binary:logistic')
model.fit(X_train, y_train)
ypred =model.predict(X_test)
auc=model.score(X_test, y_test)
pre=sklearn.metrics.precision_score( y_test, ypred)#	Compute the precision
recall=sklearn.metrics.recall_score( y_test, ypred)#	Compute the recall
print('Accuracy: %.6f percision: %.6f recall: %.6f' % (auc,pre,recall))
cnf_matrix_lr=confusion_matrix(y_test, ypred)
cnf_matrix_lr

Accuracy: 0.935498 percision: 0.527586 recall: 0.171525


/usr/local/lib/python3.5/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/usr/local/lib/python3.5/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


array([[25104,   274],
       [ 1478,   306]])

In [ ]:
结论：自己理解信用卡背景更关注召回率，于是在用网格搜索找参数时，将召回率设置为目标得分函数，找出了最优参数，在验证集上预测，比较之前的结果，
召回率也是最好的。

### 本阶段课程意见反馈题(非必答，不送分)
#### 请同学围绕以下两点进行回答：
- 自身总结：请您对您自己在本周课程的学习，收获，技能掌握等方面进行一次总结 ，也包括有哪些方面存在哪些不足，欠缺，困惑。作为将来回顾学习路径时的依据。


- 课程反馈：请就知识点，进度，难易度，教学方式，考试方式及难易度等方面向我们反馈，督促我们进行更有效的改进。

<hr>
<center><h1>####答卷结束####</h1></center>